In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [6]:
train_dataset = datasets.ImageFolder(
    r'C:\Radiologist_helper\datasets\chest_xray\train',
    transform=train_transform
)

val_dataset = datasets.ImageFolder(r'C:\Radiologist_helper\datasets\chest_xray\val', transform=val_test_transform)
test_dataset = datasets.ImageFolder(r'C:\Radiologist_helper\datasets\chest_xray\test', transform=val_test_transform)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)



In [9]:

learning_rate = 1e-4
num_classes = 2

In [8]:
class XrayResNet50(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')

        # Freeze all layers
        for param in self.model.parameters():
            param.requires_grad = False

        # Optional: unfreeze last block (layer4)
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        # Replace fully connected layer
        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [10]:
model = XrayResNet50(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

In [11]:

def train_model(model, criterion, optimizer, epochs=10, patience=5, save_path=r"C:\Radiologist_helper\trained_models\chest_xray_model.pth"):
    start = time.time()
    best_acc = 0.0
    best_loss = float('inf')
    patience_counter = 0

    all_labels, all_predictions = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            if (batch_num + 1) % 10 == 0:
                print(f"Batch: {batch_num+1}, Epoch: {epoch+1}, Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}] - Avg Train Loss: {epoch_loss:.4f}")

        # ---------- Validation ----------
        model.eval()
        correct, total = 0, 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_acc = 100 * correct / total
        print(f"Validation Loss: {val_loss:.4f} | Accuracy: {val_acc:.2f}%")

        # --- Early stopping and save ---
        if val_loss < best_loss:
            best_loss = val_loss
            best_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved at epoch {epoch+1}")
        else:
            patience_counter += 1
            print(f"No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                break

    end = time.time()
    print(f"\nTraining complete in {(end - start):.2f}s")
    print(f"Best Validation Accuracy: {best_acc:.2f}%")

    return all_labels, all_predictions

In [13]:
all_labels, all_predictions = train_model(model, criterion, optimizer, epochs=10)

Batch: 10, Epoch: 1, Loss: 0.4983
Batch: 20, Epoch: 1, Loss: 0.3323
Batch: 30, Epoch: 1, Loss: 0.2780
Batch: 40, Epoch: 1, Loss: 0.2242
Batch: 50, Epoch: 1, Loss: 0.1196
Batch: 60, Epoch: 1, Loss: 0.2007
Batch: 70, Epoch: 1, Loss: 0.0700
Batch: 80, Epoch: 1, Loss: 0.2682
Batch: 90, Epoch: 1, Loss: 0.2399
Batch: 100, Epoch: 1, Loss: 0.1335
Batch: 110, Epoch: 1, Loss: 0.0602
Batch: 120, Epoch: 1, Loss: 0.0991
Batch: 130, Epoch: 1, Loss: 0.0931
Batch: 140, Epoch: 1, Loss: 0.1119
Batch: 150, Epoch: 1, Loss: 0.2204
Batch: 160, Epoch: 1, Loss: 0.1459
Epoch [1/10] - Avg Train Loss: 0.2128
Validation Loss: 0.3583 | Accuracy: 75.00%
Best model saved at epoch 1
Batch: 10, Epoch: 2, Loss: 0.1033
Batch: 20, Epoch: 2, Loss: 0.1095
Batch: 30, Epoch: 2, Loss: 0.2245
Batch: 40, Epoch: 2, Loss: 0.0524
Batch: 50, Epoch: 2, Loss: 0.1504
Batch: 60, Epoch: 2, Loss: 0.1176
Batch: 70, Epoch: 2, Loss: 0.1957
Batch: 80, Epoch: 2, Loss: 0.0459
Batch: 90, Epoch: 2, Loss: 0.1023
Batch: 100, Epoch: 2, Loss: 0.0808

In [14]:
from sklearn.metrics import classification_report

report = classification_report(all_labels, all_predictions)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.76      0.87        80
           1       0.81      1.00      0.89        80

    accuracy                           0.88       160
   macro avg       0.90      0.88      0.88       160
weighted avg       0.90      0.88      0.88       160



In [16]:
torch.save(model.state_dict(), r'C:\Radiologist_helper\trained_models\chest_xray_model.pth')
print("Model saved successfully!")

Model saved successfully!


In [15]:
def evaluate_test(model, test_loader, model_path=r'C:\Radiologist_helper\trained_models\chest_xray_model.pth'):
    device = next(model.parameters()).device
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).int()

            total += labels.size(0)
            correct += (preds == labels.int()).sum().item()

    test_acc = 100 * correct / total
    print(f"*** Test Accuracy: {test_acc:.2f}% ***")
    return test_acc


In [18]:
test_acc = evaluate_test(model, test_loader)
print(f"*** Test Accuracy: {test_acc:.2f}% ***")

*** Test Accuracy: 99.84% ***
*** Test Accuracy: 99.84% ***
